<a href="https://colab.research.google.com/github/mathewpolonsky/CluQwA/blob/main/training%20mpnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### DeviAⁱnts

### Установка библиотек

In [ ]:
!pip install git+https://github.com/UKPLab/sentence-transformers
!pip install xformers

---
### Импорт библиотек

In [ ]:
import os
import json
from pathlib import Path

from datetime import datetime

import pandas as pd
import numpy as np

import torch
from torch.utils.data import DataLoader

from sentence_transformers import SentenceTransformer, losses, InputExample, evaluation, models, util

from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

from sklearn.model_selection import train_test_split

# from wordcloud import WordCloud

torch.cuda.is_available()

True

---
### Предобработка датасета

In [ ]:
!gdown https://lodmedia.hb.bizmrg.com/case_files/1001711/train_dataset_%D0%9C%D0%BE%D0%B9%20%D0%B3%D0%BE%D0%BB%D0%BE%D1%81.zip

Downloading...
From: https://lodmedia.hb.bizmrg.com/case_files/1001711/train_dataset_%D0%9C%D0%BE%D0%B9%20%D0%B3%D0%BE%D0%BB%D0%BE%D1%81.zip
To: /content/train_dataset_%D0%9C%D0%BE%D0%B9%20%D0%B3%D0%BE%D0%BB%D0%BE%D1%81.zip
100% 607k/607k [00:01<00:00, 516kB/s]


In [ ]:
def extract_answers(x):
    answer = x["corrected"] if "corrected" in x else x["answer"]
    cluster = x["cluster"] if "cluster" in x else ""

    return answer, cluster


def working_count(data_json):
    '''
    Правильно считает counts
    '''
    answers = data_json["answers"]

    new_answers = {}

    for answer in answers:
        if answer["answer"] not in new_answers:
            new_answers[answer["answer"]] = answer

        else:
            # print(new_answers[answer["answer"]])
            new_answers[answer["answer"]]["count"] += answer["count"]

    return list(new_answers.values())

In [ ]:
data = pd.DataFrame(columns=["question_id", "answer1_id", "answer2_id", "answer1", "answer2", "target"])

path = Path(r"train_dataset_Мой голос")
path = path / "labeled"

file_names = sorted(os.listdir(path))

answers1, answers2 = [], []
answers_id1, answers_id2 = [], []
targets, questions_id = [], []

for name in file_names[:int(len(file_names)* 0.75)]:
    try:
        with open(path / f"{name}", "r") as file:
            data_json = json.load(file)

        data_json["answers"] = working_count(data_json)

    except json.JSONDecodeError:
        print(f"{name} не работает")

    answers = data_json["answers"]

    for i in range(len(answers) - 1):
        for j in range(i+1, len(answers)):
            if (answers[i]["cluster"] == answers[j]["cluster"]) or (
                (answers[i]["cluster"] != answers[j]["cluster"]) and np.random.random() < 0.05):

                answers1.append(answers[i]["answer"])
                answers2.append(answers[j]["answer"])

                targets.append(
                    int(answers[i]["cluster"] == answers[j]["cluster"]))

                answers_id1.append(i)
                answers_id2.append(j)

                questions_id.append(data_json["id"])


data["answer1"] = answers1
data["answer2"] = answers2

data["target"] = targets

data["answer1_id"] = answers_id1
data["answer2_id"] = answers_id2

data["question_id"] = questions_id

data

,question_id,answer1_id,answer2_id,answer1,answer2,target
0,19749,1,3,сократить дублирование,убедить работника что его жиз,0
1,19749,5,6,уменьшение бюрократии,упростить бюрократич. нагрузку,1
2,19750,3,7,узконаправленная деятельность,физическая,0
3,19750,8,9,эта поддержка уже есть,методическая,0
4,19750,9,10,методическая,методология,1
...,...,...,...,...,...,...
5658,26608,31,32,санкт - петербург,санкт-петербург,1
5659,26608,31,42,санкт - петербург,электросталь лучший город,0
5660,26608,37,38,трехгорный,трёхгорный,1
5661,26608,38,40,трёхгорный,шумиха,0


---
### Clustering

#### train

In [ ]:
train = data.apply(
    lambda x: InputExample(texts=[x['answer1'], x['answer2']], label=x['target']), axis=1
    ).tolist()

train_dataloader = DataLoader(train, shuffle=True, batch_size=64)

In [ ]:
# model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
model.to('cuda')

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
)

In [ ]:
num_epochs = 1
warmup_steps = 1000
# model_save_path = 'models/minilm_' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_save_path = 'models/mpnet_0.5_1ep'
distance_metric = losses.SiameseDistanceMetric.COSINE_DISTANCE
margin = .5

train_loss = losses.OnlineContrastiveLoss(model=model, distance_metric=distance_metric, margin=margin)
model_save_path

'models/mpnet_0.5_1ep'

In [ ]:
model = torch.compile(model)

In [ ]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    # evaluator=evaluator,
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    output_path=model_save_path
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/89 [00:00<?, ?it/s]

#### val

In [ ]:
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
model.to('cuda')

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
)

In [ ]:
path = Path(r"train_dataset_Мой голос")
path = path / "labeled"

In [ ]:
def metric(data_json, threshold=.9):
    data_json["answers"] = working_count(data_json)

    data = [extract_answers(i) for i in data_json["answers"]]

    corpus = np.array([i[0] for i in data])
    true_clusters = np.array([i[1] for i in data])

    answers = data_json["answers"]

    codes = model.encode(corpus, convert_to_tensor=True, show_progress_bar=False)

    clusters = util.community_detection(codes, min_community_size=1, threshold=threshold)

    # print(f"Вопрос: {data_json['question']}\n")

    full_text = ""

    for cluster in clusters:
        # print("\n".join([text + ' | ' + cluster for (text, cluster) in zip(corpus[cluster], true_clusters[cluster])]), end="\n-------------------\n")
        full_text += " ".join(corpus[cluster]) + '\n'

    cluster_dict = {}
    m = -1
    clusters_num = []

    for answer in answers:
        if answer["cluster"] not in cluster_dict:
            m += 1
            cluster_dict[answer["cluster"]] = m
            clusters_num.append(m)
        else:
            clusters_num.append(cluster_dict[answer["cluster"]])

    clusters_num = np.array(clusters_num)

    score = 0
    score_match = 0

    for i in range(m):
        for j in range(len(clusters)):
            if set(np.where(clusters_num == i)[0]) == (clusters[j]):
                # print(clusters_num, i, clusters[j])
                score_match += 1
                score += 1
                break

            if set(np.where(clusters_num == i)[0]).issubset(clusters[j]):
                # print(clusters_num, i, clusters[j])
                score += 1
                break

    return score / m, score_match / m, len(clusters)

In [ ]:
file_names = sorted(os.listdir(path))
file_names = file_names[int(len(file_names)* 0.75):]


for threshold in np.arange(0.35, 1.0, 0.1):
    metrics = []
    # metrics_match = []
    mnms = []

    for file in file_names:
        try:
            with open(path / f"{file}", "r") as file:
                data_json = json.load(file)

            result = metric(data_json, threshold)

            metrics.append(result[0])
            # metrics_match.append(metric(data_json)[1])
            mnms.append(result[2])

        except json.JSONDecodeError:
            print(f"{name} не работает")

    print(threshold, np.mean(metrics), mnms)

26608.json не работает
0.35 0.9909659372176648 [1, 3, 2, 2, 1, 2, 1, 1, 1, 1, 9, 2, 1, 5, 3, 1, 1, 1, 1, 1, 2, 5, 1, 1, 1, 1, 1, 1, 2, 4, 1, 2, 2, 2, 3, 2, 2, 1, 1]
26608.json не работает
0.44999999999999996 0.9646995697542159 [6, 9, 5, 2, 4, 6, 1, 2, 1, 3, 13, 4, 3, 25, 15, 1, 7, 1, 2, 2, 6, 16, 4, 3, 1, 1, 2, 1, 7, 7, 1, 10, 8, 6, 9, 6, 5, 1, 4]
26608.json не работает
0.5499999999999999 0.9223080789033239 [18, 26, 11, 6, 8, 11, 5, 6, 4, 10, 21, 7, 7, 67, 46, 4, 20, 2, 5, 2, 15, 28, 11, 5, 2, 3, 3, 2, 18, 23, 4, 30, 20, 9, 14, 13, 8, 4, 5]
26608.json не работает
0.6499999999999999 0.9096275408105728 [41, 52, 26, 10, 13, 12, 10, 13, 10, 19, 24, 9, 15, 125, 79, 11, 39, 4, 6, 4, 22, 48, 27, 10, 5, 11, 6, 3, 38, 37, 6, 38, 27, 17, 18, 24, 15, 13, 7]
26608.json не работает
0.7499999999999999 0.8976347186208216 [61, 85, 39, 14, 21, 20, 16, 20, 27, 37, 25, 15, 35, 174, 115, 24, 70, 6, 7, 4, 31, 64, 42, 13, 8, 21, 11, 3, 48, 56, 7, 47, 38, 21, 19, 35, 21, 26, 11]
26608.json не работает
0.8499